In [182]:
import sys
sys.path.append('/home/cham/data/ToolFun/python/')

In [183]:
from astropy.table import Table
from spectrum import lamost_filepath,measure_line_index_null_result,measure_line_index
from IPython.parallel import require

In [10]:
catalogpath = '/pool/LAMOST/DR2/calalog/dr2_original/dr2c.fits'
catalog = Table.read(catalogpath)
filepath = []
for i in xrange(len(catalog)):
    filepath.append(lamost_filepath(\
                            catalog['planid'][i],\
                            catalog['mjd'][i],\
                            catalog['spid'][i],\
                            catalog['fiberid'][i]))

In [167]:
from IPython.parallel import parallel,Client
rc = Client()
dview = rc[:]

In [152]:
@dview.remote(block=True)
def measure_line_index_loopfun_start():
    import sys
    sys.path.append('/home/cham/data/ToolFun/python/')
    #from spectrum import measure_line_index_loopfun
    return sys.path

In [178]:
#@require('measure_line_index')
def measure_line_index_loopfun(filepath):
    num_refit = 100
    line_info_dib5780 = {\
#                'line_center':5780,\
                'line_range':(5775,5785),\
                'line_shoulder_left':(5755,5775),\
                'line_shoulder_right':(5805,5825)}
    line_info_dib5797 = {\
#                'line_center':5797,\
                'line_range':(5792,5802),\
                'line_shoulder_left':(5755,5775),\
                'line_shoulder_right':(5805,5825)}            
    line_info_dib6284 = {\
#                'line_center':6285,\
                'line_range':(6280,6290),\
                'line_shoulder_left':(6260,6280),\
                'line_shoulder_right':(6310,6330)}
#    try:
    # read spectrum +++++++++++++++++++++++++++++++++++++++++++++++++++
    spec = read_spectrum(filepath,'lamost_dr2')
    # measure DIBs ++++++++++++++++++++++++++++++++++++++++++++++++++++
    # DIB5780
    line_indx_dib5780 = measure_line_index(\
                            wave=spec['wave'],\
                            flux=spec['flux'],\
                            flux_err=spec['flux_err'],\
                            mask=spec['and_mask'],\
                            line_info=line_info_dib5780,\
                            num_refit=num_refit,z=0,\
                            )
    # DIB5797
    line_indx_dib5797 = measure_line_index(\
                            wave=spec['wave'],\
                            flux=spec['flux'],\
                            flux_err=spec['flux_err'],\
                            mask=spec['and_mask'],\
                            line_info=line_info_dib5797,\
                            num_refit=num_refit,z=0,\
                            )
    # DIB6284 
    line_indx_dib6284 = measure_line_index(\
                            wave=spec['wave'],\
                            flux=spec['flux'],\
                            flux_err=spec['flux_err'],\
                            mask=spec['and_mask'],\
                            line_info=line_info_dib6284,\
                            num_refit=num_refit,z=0,\
                            )
    return line_indx_dib5780,line_indx_dib5797,line_indx_dib6284

In [179]:
dview['filepath[0]']

['/pool/LAMOST/DR2/spectra/fits/F5902/spec-55859-F5902_sp01-001.fits',
 '/pool/LAMOST/DR2/spectra/fits/GAC_083N28_M1/spec-55899-GAC_083N28_M1_sp15-177.fits',
 '/pool/LAMOST/DR2/spectra/fits/B5592204/spec-55922-B5592204_sp07-183.fits',
 '/pool/LAMOST/DR2/spectra/fits/F5595003/spec-55950-F5595003_sp11-116.fits',
 '/pool/LAMOST/DR2/spectra/fits/F5597308/spec-55973-F5597308_sp13-119.fits',
 '/pool/LAMOST/DR2/spectra/fits/VB3_210N09_V1/spec-56062-VB3_210N09_V1_sp11-188.fits',
 '/pool/LAMOST/DR2/spectra/fits/EG220307N031056B02/spec-56207-EG220307N031056B02_sp09-211.fits',
 '/pool/LAMOST/DR2/spectra/fits/M31007N36B2/spec-56232-M31007N36B2_sp13-043.fits',
 '/pool/LAMOST/DR2/spectra/fits/VB054N40V1/spec-56258-VB054N40V1_sp04-191.fits',
 '/pool/LAMOST/DR2/spectra/fits/HD114225N022143B/spec-56286-HD114225N022143B_sp15-248.fits',
 '/pool/LAMOST/DR2/spectra/fits/GAC094N27M1/spec-56304-GAC094N27M1_sp02-158.fits',
 '/pool/LAMOST/DR2/spectra/fits/GAC101N53V2/spec-56326-GAC101N53V2_sp06-212.fits',
 '/p

In [68]:
dview.scatter('filepath',filepath)

<AsyncResult: scatter>

In [180]:
results_async = dview.map_async(measure_line_index_loopfun,filepath[0:10])

In [181]:
results_async.get()

[({'EW_int': 0.21945088941428112,
   'EW_int_err': 0.32208192223007431,
   'SN_local_flux_err': 9.3267247858169533,
   'SN_local_flux_std': 11.863608929339625,
   'flag_good_pixel': True,
   'mod_gauss_amplitude': 3.8274120663084328,
   'mod_gauss_amplitude_err': nan,
   'mod_gauss_amplitude_std': 1.4279852558735989e+23,
   'mod_gauss_center': 5785.8190138531072,
   'mod_gauss_center_err': nan,
   'mod_gauss_center_std': 149.96861642426319,
   'mod_gauss_sigma': 0.33939707178611567,
   'mod_gauss_sigma_err': nan,
   'mod_gauss_sigma_std': 16.990466160315869,
   'mod_linear_intercept': 320.22404912841228,
   'mod_linear_intercept_err': 371.54247251064959,
   'mod_linear_slope': -0.037943228241089054,
   'mod_linear_slope_err': 0.064178510614486303},
  {'EW_int': -2.6060270900082427,
   'EW_int_err': 0.25790686812646169,
   'SN_local_flux_err': 9.3267247858169533,
   'SN_local_flux_std': 11.863608929339625,
   'flag_good_pixel': True,
   'mod_gauss_amplitude': -1.6519693314544015,
   'mo

In [12]:

dview.execute('sys.path.append(\'/home/cham/data/ToolFun/python\')').get()
dview.execute('sys.path').get()

results_async = dview.map_async(measure_line_index_loopfun,filepath[0])
results = results_async.get()

In [ ]:
results_async = dview.map_async(measure_line_index_loopfun,filepath[0])
results = results_async.get()